# RQ2: SALES FORECASTING - VAR/VECM IMPLEMENTATION
## Complete Pipeline with All Assumption Tests

---

### 📋 Checklist Tahapan VAR/VECM:

1. ✅ **Data Preparation** - Aggregate revenue per product, add sentiment from reviews
2. ✅ **Uji Stasioneritas (ADF/KPSS)** - Check I(0) or I(1)
3. ✅ **Uji Lag Optimum (AIC/BIC/HQ)** - Determine optimal lag
4. ✅ **Uji Kointegrasi (Johansen)** - Check cointegration for VECM
5. ✅ **Estimasi Model (VAR/VECM)** - Fit appropriate model
6. ✅ **Uji Stabilitas (Eigenvalue)** - Check model stability
7. ✅ **Uji Diagnostik Residual**:
   - Autokorelasi (LM Test)
   - Normalitas (Jarque-Bera)
   - Heteroskedastisitas (ARCH Test)
8. ✅ **Granger Causality** - Test causal relationships
9. ✅ **IRF & FEVD** - Dynamic analysis
10. ✅ **Forecasting** - Future predictions

---

## 1. Setup and Data Loading

Assuming you've already loaded the data in previous cells, we'll continue from there.

In [ ]:
# Import additional libraries needed for RQ2
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox
from statsmodels.tsa.stattools import kpss
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

print("✅ Libraries loaded for RQ2 analysis")

## 2. Data Preparation for VAR/VECM

Prepare time series data with:
- Daily revenue aggregation per product
- Sentiment scores from reviews
- Rating averages
- Marketing spend (if applicable)

In [ ]:
# Create time series for VAR/VECM analysis
def prepare_var_data(sales, reviews, products, marketing=None):
    """
    Prepare multivariate time series for VAR/VECM
    Includes revenue, sentiment, and other features
    """
    print("="*80)
    print("DATA PREPARATION FOR VAR/VECM")
    print("="*80)
    
    # 1. Aggregate daily revenue per product
    daily_revenue = sales.groupby(['date', 'product_id'])['revenue'].sum().reset_index()
    revenue_ts = daily_revenue.pivot(index='date', columns='product_id', values='revenue')
    revenue_ts = revenue_ts.fillna(0)
    revenue_ts.columns = [f'P{col}' for col in revenue_ts.columns]
    
    print(f"✓ Revenue time series created: {revenue_ts.shape}")
    
    # 2. Add sentiment analysis from reviews
    analyzer = SentimentIntensityAnalyzer()
    
    reviews_copy = reviews.copy()
    reviews_copy['sentiment'] = reviews_copy['review'].apply(
        lambda x: analyzer.polarity_scores(str(x))['compound'] if pd.notna(x) else 0
    )
    
    # Aggregate daily sentiment
    daily_sentiment = reviews_copy.groupby(['date', 'product_id']).agg({
        'sentiment': 'mean',
        'rating': 'mean',
        'review_id': 'count'  # Review volume
    }).reset_index()
    daily_sentiment.rename(columns={'review_id': 'review_count'}, inplace=True)
    
    # Create sentiment time series
    sentiment_ts = daily_sentiment.pivot(index='date', columns='product_id', values='sentiment')
    sentiment_ts = sentiment_ts.fillna(0)
    sentiment_ts.columns = [f'sent_P{col}' for col in sentiment_ts.columns]
    
    # Create rating time series
    rating_ts = daily_sentiment.pivot(index='date', columns='product_id', values='rating')
    rating_ts = rating_ts.fillna(method='ffill').fillna(3.0)  # Fill with neutral rating
    rating_ts.columns = [f'rating_P{col}' for col in rating_ts.columns]
    
    print(f"✓ Sentiment features added: {sentiment_ts.shape[1]} variables")
    print(f"✓ Rating features added: {rating_ts.shape[1]} variables")
    
    # 3. Combine all features
    combined_ts = revenue_ts.copy()
    
    # Option to include sentiment and rating as exogenous variables
    # For now, we'll focus on revenue only for main VAR/VECM
    
    print(f"\n📊 Final time series shape: {combined_ts.shape}")
    print(f"Variables: {list(combined_ts.columns)}")
    
    return combined_ts, sentiment_ts, rating_ts

# Prepare data
revenue_ts, sentiment_ts, rating_ts = prepare_var_data(sales, reviews, products, marketing)

## 3. Exploratory Analysis of Time Series

In [ ]:
# Visualize all 15 product revenue trends
fig = make_subplots(
    rows=5, cols=3,
    subplot_titles=[f'Product {i}' for i in range(1, 16)],
    vertical_spacing=0.06,
    horizontal_spacing=0.08
)

for i, col in enumerate(revenue_ts.columns[:15], 1):
    row = (i-1) // 3 + 1
    col_idx = (i-1) % 3 + 1
    
    fig.add_trace(
        go.Scatter(
            x=revenue_ts.index,
            y=revenue_ts[col],
            mode='lines',
            name=col,
            line=dict(width=1),
            showlegend=False
        ),
        row=row, col=col_idx
    )

fig.update_layout(
    height=1000,
    title_text="Daily Revenue Trends - 15 Products",
    showlegend=False
)
fig.update_xaxes(tickformat='%Y-%m')
fig.update_yaxes(tickformat='$,.0f')
fig.show()

# Summary statistics
print("\n📊 Revenue Statistics Summary:")
summary = revenue_ts.describe().T[['mean', 'std', 'min', 'max']]
summary['CV'] = summary['std'] / summary['mean']
print(summary.sort_values('mean', ascending=False))

## 4. TAHAP 1: Uji Stasioneritas (ADF & KPSS)

In [ ]:
def stationarity_test_complete(df):
    """
    Complete stationarity testing with ADF and KPSS
    """
    print("="*80)
    print("TAHAP 1: UJI STASIONERITAS (ADF & KPSS)")
    print("="*80)
    
    results = []
    
    for col in df.columns[:15]:  # Test all 15 products
        # ADF Test
        adf_result = adfuller(df[col].dropna(), autolag='AIC')
        
        # KPSS Test
        kpss_result = kpss(df[col].dropna(), regression='c', nlags='auto')
        
        # Determine order of integration
        if adf_result[1] < 0.05 and kpss_result[1] > 0.05:
            order = 'I(0)'
            status = '✅ Stationary'
        elif adf_result[1] > 0.05 and kpss_result[1] < 0.05:
            order = 'I(1)'
            status = '❌ Non-stationary'
        else:
            order = 'Mixed'
            status = '⚠️ Mixed results'
        
        results.append({
            'Product': col,
            'ADF_stat': adf_result[0],
            'ADF_pval': adf_result[1],
            'KPSS_stat': kpss_result[0],
            'KPSS_pval': kpss_result[1],
            'Order': order,
            'Status': status
        })
    
    # Create DataFrame
    results_df = pd.DataFrame(results)
    
    # Display results
    print("\nStationarity Test Results:")
    print("-"*80)
    print(results_df[['Product', 'ADF_pval', 'KPSS_pval', 'Order', 'Status']].to_string(index=False))
    
    # Summary
    n_stationary = (results_df['Order'] == 'I(0)').sum()
    n_nonstationary = (results_df['Order'] == 'I(1)').sum()
    n_mixed = (results_df['Order'] == 'Mixed').sum()
    
    print(f"\n📊 Summary:")
    print(f"  I(0) Stationary: {n_stationary}/15")
    print(f"  I(1) Non-stationary: {n_nonstationary}/15")
    print(f"  Mixed results: {n_mixed}/15")
    
    # Decision
    if n_nonstationary > 7:
        print(f"\n⚠️ Majority non-stationary → Test for cointegration (potential VECM)")
        use_vecm = True
    else:
        print(f"\n✅ Majority stationary → Use VAR model")
        use_vecm = False
    
    return results_df, use_vecm

# Test stationarity
stationarity_results, potential_vecm = stationarity_test_complete(revenue_ts)

## 5. TAHAP 2: Uji Lag Optimum (AIC/BIC/HQ)

In [ ]:
# Split data for training and testing
test_size = 30
train_data = revenue_ts[:-test_size]
test_data = revenue_ts[-test_size:]

print(f"Data Split:")
print(f"  Training: {len(train_data)} days")
print(f"  Testing: {len(test_data)} days")

def select_optimal_lag(df, maxlags=15):
    """
    Select optimal lag using information criteria
    """
    print("\n" + "="*80)
    print("TAHAP 2: UJI LAG OPTIMUM (AIC/BIC/HQ)")
    print("="*80)
    
    model = VAR(df)
    lag_selection = model.select_order(maxlags=maxlags)
    
    print("\nLag Selection Table:")
    print(lag_selection.summary())
    
    # Get optimal lags
    optimal_lags = {
        'AIC': lag_selection.aic,
        'BIC': lag_selection.bic,
        'FPE': lag_selection.fpe,
        'HQIC': lag_selection.hqic
    }
    
    print("\n📊 Optimal Lag by Criterion:")
    for criterion, lag in optimal_lags.items():
        print(f"  {criterion}: {lag}")
    
    # Use AIC
    optimal_lag = optimal_lags['AIC']
    print(f"\n✅ Selected optimal lag: {optimal_lag} (based on AIC)")
    
    return optimal_lag, lag_selection

optimal_lag, lag_info = select_optimal_lag(train_data)

## 6. TAHAP 3: Uji Kointegrasi (Johansen Test)

In [ ]:
def johansen_cointegration_test(df, lag):
    """
    Johansen cointegration test
    """
    print("="*80)
    print("TAHAP 3: UJI KOINTEGRASI (JOHANSEN TEST)")
    print("="*80)
    
    # Perform test
    result = coint_johansen(df, det_order=0, k_ar_diff=lag)
    
    # Extract statistics
    trace_stat = result.lr1
    cv_trace = result.cvt  # Critical values
    max_eigen = result.lr2
    cv_eigen = result.cvm
    
    print("\nTrace Test Results:")
    print("-"*80)
    print(f"{'H0: r<=':<10} {'Trace':<15} {'90% CV':<15} {'95% CV':<15} {'99% CV':<15} {'Result'}")
    print("-"*80)
    
    n_coint = 0
    for i in range(len(trace_stat)):
        if trace_stat[i] > cv_trace[i, 1]:  # 95% level
            result_str = "Reject H0 ✓"
            n_coint = i + 1
        else:
            result_str = "Fail to reject"
        
        print(f"{i:<10} {trace_stat[i]:<15.2f} {cv_trace[i,0]:<15.2f} "
              f"{cv_trace[i,1]:<15.2f} {cv_trace[i,2]:<15.2f} {result_str}")
    
    print(f"\n📊 Cointegration Results:")
    print(f"  Number of cointegrating vectors: {n_coint}")
    
    if n_coint > 0:
        print(f"  ✅ Cointegration detected → Use VECM with rank={n_coint}")
        use_vecm = True
    else:
        print(f"  ❌ No cointegration → Use VAR model")
        use_vecm = False
    
    return n_coint, use_vecm, result

if potential_vecm:
    coint_rank, use_vecm, johansen_result = johansen_cointegration_test(train_data, optimal_lag)
else:
    print("\n✅ Skipping cointegration test (data is stationary)")
    use_vecm = False
    coint_rank = 0

## 7. TAHAP 4: Estimasi Model (VAR atau VECM)

In [ ]:
print("="*80)
print("TAHAP 4: ESTIMASI MODEL")
print("="*80)

if use_vecm and coint_rank > 0:
    print(f"\n📈 Estimating VECM Model")
    print(f"  Cointegration rank: {coint_rank}")
    print(f"  Lag order: {optimal_lag-1}")
    
    # Fit VECM
    vecm_model = VECM(train_data, k_ar_diff=optimal_lag-1, coint_rank=coint_rank, deterministic='ci')
    vecm_fit = vecm_model.fit()
    
    # Get forecast
    forecast = vecm_fit.predict(steps=len(test_data))
    forecast_df = pd.DataFrame(forecast, index=test_data.index, columns=train_data.columns)
    
    model = vecm_fit
    model_type = 'VECM'
    
    print("\nVECM Model Summary:")
    print(f"  Number of equations: {vecm_fit.neqs}")
    print(f"  Sample size: {vecm_fit.nobs}")
    
    # Show adjustment coefficients
    print("\nAdjustment Coefficients (Speed of Adjustment):")
    alpha = vecm_fit.alpha
    for i in range(min(5, alpha.shape[0])):
        print(f"  Product {i+1}: {alpha[i,0]:.4f}")
    
else:
    print(f"\n📈 Estimating VAR Model")
    print(f"  Lag order: {optimal_lag}")
    
    # Check if differencing needed
    if potential_vecm and not use_vecm:
        print("  Note: Non-stationary but no cointegration → Using differenced data")
        train_diff = train_data.diff().dropna()
        test_diff = test_data.diff().dropna()
        
        var_model = VAR(train_diff)
        var_fit = var_model.fit(optimal_lag)
        
        # Forecast differences
        forecast_diff = var_fit.forecast(train_diff.values[-optimal_lag:], steps=len(test_diff))
        
        # Convert back to levels
        forecast = np.zeros((len(test_data), len(train_data.columns)))
        last_value = train_data.iloc[-1].values
        forecast[0] = last_value + forecast_diff[0]
        for i in range(1, len(forecast)):
            forecast[i] = forecast[i-1] + forecast_diff[i]
        
        forecast_df = pd.DataFrame(forecast, index=test_data.index, columns=train_data.columns)
        
    else:
        # Regular VAR
        var_model = VAR(train_data)
        var_fit = var_model.fit(optimal_lag)
        
        forecast = var_fit.forecast(train_data.values[-optimal_lag:], steps=len(test_data))
        forecast_df = pd.DataFrame(forecast, index=test_data.index, columns=train_data.columns)
    
    model = var_fit
    model_type = 'VAR'
    
    print("\nVAR Model Summary:")
    print(f"  Order: VAR({var_fit.k_ar})")
    print(f"  Number of equations: {var_fit.neqs}")
    print(f"  Log-likelihood: {var_fit.llf:.2f}")
    print(f"  AIC: {var_fit.aic:.2f}")
    print(f"  BIC: {var_fit.bic:.2f}")

# Calculate performance
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

mape = mean_absolute_percentage_error(test_data.values.flatten(), forecast_df.values.flatten())
rmse = np.sqrt(mean_squared_error(test_data.values.flatten(), forecast_df.values.flatten()))

print(f"\n📊 Forecast Performance:")
print(f"  MAPE: {mape:.2%}")
print(f"  RMSE: ${rmse:,.2f}")

## 8. TAHAP 5: Uji Stabilitas Model (Eigenvalue/AR Root)

In [ ]:
print("="*80)
print("TAHAP 5: UJI STABILITAS MODEL")
print("="*80)

if model_type == 'VAR':
    # Get roots
    roots = model.roots
    
    print(f"\nStability Analysis:")
    print(f"  Total roots: {len(roots)}")
    
    # Check stability
    unstable = []
    for i, root in enumerate(roots):
        if abs(root) >= 1:
            unstable.append((i, root, abs(root)))
    
    # Show first 10 roots
    print("\nCharacteristic Roots (First 10):")
    print(f"{'Index':<10} {'Root':<25} {'Modulus':<15} {'Status'}")
    print("-"*70)
    
    for i in range(min(10, len(roots))):
        modulus = abs(roots[i])
        status = "✅ Stable" if modulus < 1 else "❌ UNSTABLE"
        print(f"{i:<10} {str(roots[i]):<25} {modulus:<15.4f} {status}")
    
    max_modulus = max(abs(roots))
    is_stable = len(unstable) == 0
    
    print(f"\n📊 Stability Results:")
    print(f"  Maximum modulus: {max_modulus:.4f}")
    print(f"  Unstable roots: {len(unstable)}")
    print(f"  Model stability: {'✅ STABLE' if is_stable else '❌ UNSTABLE'}")
    
    if not is_stable:
        print("\n⚠️ Model is unstable! Consider:")
        print("  - Reducing lag order")
        print("  - Checking for structural breaks")
        print("  - Data transformation")
    
    # Plot roots
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'k-', alpha=0.3, label='Unit Circle')
    
    # Plot roots
    for root in roots:
        color = 'blue' if abs(root) < 1 else 'red'
        ax.scatter(root.real, root.imag, c=color, s=30, alpha=0.6)
    
    ax.set_xlim([-1.5, 1.5])
    ax.set_ylim([-1.5, 1.5])
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title(f'{model_type} Characteristic Roots')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
    plt.tight_layout()
    plt.show()
    
else:
    print("\nVECM Stability:")
    print("  VECM models are stable by construction when cointegration exists")
    print("  ✅ Model is stable")
    is_stable = True

## 9. TAHAP 6: Uji Diagnostik Residual

In [ ]:
print("="*80)
print("TAHAP 6: UJI DIAGNOSTIK RESIDUAL")
print("="*80)

# Get residuals
residuals = pd.DataFrame(model.resid, columns=train_data.columns)

# A. AUTOKORELASI (Ljung-Box Test)
print("\nA. AUTOKORELASI (Ljung-Box Test)")
print("-"*60)
print("Testing first 5 products:")

for i, col in enumerate(residuals.columns[:5]):
    lb_test = acorr_ljungbox(residuals[col], lags=10, return_df=True)
    min_pval = lb_test['lb_pvalue'].min()
    status = "✅ No autocorrelation" if min_pval > 0.05 else "⚠️ Autocorrelation"
    print(f"  {col}: Min p-value = {min_pval:.4f} | {status}")

# B. NORMALITAS (Jarque-Bera)
print("\nB. NORMALITAS (Jarque-Bera Test)")
print("-"*60)

if model_type == 'VAR':
    jb_test = model.test_normality()
    print(f"Multivariate test:")
    print(f"  JB statistic: {jb_test.statistic:.4f}")
    print(f"  P-value: {jb_test.pvalue:.4f}")
    print(f"  Result: {'✅ Normal' if jb_test.pvalue > 0.05 else '⚠️ Not normal'}")

print("\nIndividual tests (first 5 products):")
for col in residuals.columns[:5]:
    jb_stat, jb_pval = stats.jarque_bera(residuals[col])
    status = "✅ Normal" if jb_pval > 0.05 else "⚠️ Not normal"
    print(f"  {col}: JB = {jb_stat:.2f}, p-value = {jb_pval:.4f} | {status}")

# C. HETEROSKEDASTISITAS (ARCH Test)
print("\nC. HETEROSKEDASTISITAS (ARCH Test)")
print("-"*60)
print("Testing first 5 products:")

for col in residuals.columns[:5]:
    arch_test = het_arch(residuals[col])
    arch_stat = arch_test[0]
    arch_pval = arch_test[1]
    status = "✅ Homoskedastic" if arch_pval > 0.05 else "⚠️ Heteroskedastic"
    print(f"  {col}: ARCH = {arch_stat:.2f}, p-value = {arch_pval:.4f} | {status}")

# D. DURBIN-WATSON TEST
print("\nD. DURBIN-WATSON TEST")
print("-"*60)
print("Testing first 5 products:")

for col in residuals.columns[:5]:
    dw = durbin_watson(residuals[col])
    if 1.5 < dw < 2.5:
        status = "✅ No serial correlation"
    elif dw < 1.5:
        status = "⚠️ Positive serial correlation"
    else:
        status = "⚠️ Negative serial correlation"
    print(f"  {col}: DW = {dw:.3f} | {status}")

## 10. TAHAP 7: Granger Causality Test

In [ ]:
print("="*80)
print("TAHAP 7: UJI KAUSALITAS (GRANGER CAUSALITY)")
print("="*80)
print("Testing for cannibalization: New products (P13-P15) → Old products (P1-P5)")
print("-"*60)

# Test new products against old products
new_products = ['P13', 'P14', 'P15']
old_products = ['P1', 'P2', 'P3', 'P4', 'P5']

granger_results = []

for new_prod in new_products:
    for old_prod in old_products[:3]:  # Test against first 3
        try:
            # Prepare data
            test_pair = train_data[[old_prod, new_prod]].dropna()
            
            # Granger test
            gc_test = grangercausalitytests(test_pair, maxlag=5, verbose=False)
            
            # Get minimum p-value
            p_values = [gc_test[lag][0]['ssr_ftest'][1] for lag in range(1, 6)]
            min_pval = min(p_values)
            best_lag = p_values.index(min_pval) + 1
            
            causality = "YES ✓" if min_pval < 0.05 else "NO ✗"
            
            granger_results.append({
                'New': new_prod,
                'Old': old_prod,
                'Min_pval': min_pval,
                'Lag': best_lag,
                'Causes': causality
            })
            
            print(f"  {new_prod} → {old_prod}: p={min_pval:.4f} (lag {best_lag}) | {causality}")
            
        except:
            print(f"  {new_prod} → {old_prod}: Test failed")

# Summary
granger_df = pd.DataFrame(granger_results)
n_significant = (granger_df['Causes'] == 'YES ✓').sum() if len(granger_df) > 0 else 0

print(f"\n📊 Granger Causality Summary:")
print(f"  Significant relationships: {n_significant}/{len(granger_df)}")

if n_significant > 0:
    print("  ⚠️ Evidence of potential cannibalization!")
else:
    print("  ✅ No strong evidence of cannibalization")

## 11. TAHAP 8: Impulse Response Function (IRF)

In [ ]:
print("="*80)
print("TAHAP 8: IMPULSE RESPONSE FUNCTION (IRF)")
print("="*80)

if model_type == 'VAR':
    # Calculate IRF
    irf = model.irf(10)
    
    print("\nImpulse Response Analysis:")
    print("How shocks propagate across products")
    print("-"*60)
    
    # Get IRF values
    irf_values = irf.irfs
    
    # Analyze key relationships
    print("\nResponse to shock from Product 1:")
    for j in range(min(5, irf_values.shape[1])):
        max_response = np.max(np.abs(irf_values[:, j, 0]))
        peak_period = np.argmax(np.abs(irf_values[:, j, 0]))
        print(f"  Product {j+1}: Max response = {max_response:.4f} at period {peak_period}")
    
    # Plot IRF for key products
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    
    for i in range(3):
        for j in range(3):
            idx = i * 3 + j
            if idx < 9:
                # Response of product idx to shock from product 0
                axes[i, j].plot(irf_values[:, idx, 0], 'b-', linewidth=2)
                axes[i, j].axhline(y=0, color='k', linestyle='--', alpha=0.3)
                axes[i, j].set_title(f'Response of P{idx+1} to P1 shock')
                axes[i, j].set_xlabel('Periods')
                axes[i, j].set_ylabel('Response')
                axes[i, j].grid(True, alpha=0.3)
    
    plt.suptitle('Impulse Response Functions - Product Interactions', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Cumulative IRF
    print("\nCumulative Impulse Response (10 periods):")
    cum_irf = irf.cum_effects(10)
    
    for j in range(min(5, cum_irf.shape[1])):
        cum_effect = cum_irf[9, j, 0]  # Period 10, from product 1
        print(f"  Product {j+1}: Cumulative effect = {cum_effect:.4f}")
    
else:
    print("IRF analysis for VECM requires specialized implementation")
    print("Skipping for now...")

## 12. TAHAP 9: Forecast Error Variance Decomposition (FEVD)

In [ ]:
print("="*80)
print("TAHAP 9: FORECAST ERROR VARIANCE DECOMPOSITION (FEVD)")
print("="*80)

if model_type == 'VAR':
    # Calculate FEVD
    fevd = model.fevd(10)
    
    print("\nVariance Decomposition Analysis:")
    print("Which products explain variance in others")
    print("-"*60)
    
    # Get decomposition
    decomp = fevd.decomp
    
    # Analyze for first 5 products
    for i in range(min(5, decomp.shape[0])):
        print(f"\nProduct {i+1} variance explained by (at period 10):")
        
        # Get contributions at period 10
        contributions = decomp[i, 9, :]
        
        # Sort by contribution
        sorted_idx = np.argsort(contributions)[::-1]
        
        # Show top 3 contributors
        for j in sorted_idx[:3]:
            print(f"  Product {j+1}: {contributions[j]*100:.1f}%")
    
    # Visualize FEVD
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    for idx in range(4):
        if idx < decomp.shape[0]:
            row = idx // 2
            col = idx % 2
            
            # Plot stacked area for top 5 contributors
            data = decomp[idx, :, :5].T * 100  # Convert to percentage
            
            axes[row, col].stackplot(range(10), data,
                                   labels=[f'P{i+1}' for i in range(5)],
                                   alpha=0.7)
            axes[row, col].set_xlabel('Periods')
            axes[row, col].set_ylabel('Variance Share (%)')
            axes[row, col].set_title(f'FEVD: Product {idx+1}')
            axes[row, col].legend(loc='right', fontsize=8)
            axes[row, col].grid(True, alpha=0.3)
    
    plt.suptitle('Forecast Error Variance Decomposition', fontsize=14)
    plt.tight_layout()
    plt.show()
    
else:
    print("FEVD analysis for VECM requires specialized implementation")
    print("Skipping for now...")

## 13. Future Forecast (30 days ahead)

In [ ]:
print("="*80)
print("FUTURE FORECAST (Next 30 Days)")
print("="*80)

# Refit model on all data for future forecast
all_data = pd.concat([train_data, test_data])

if model_type == 'VAR':
    # Refit VAR
    final_model = VAR(all_data)
    final_fit = final_model.fit(optimal_lag)
    
    # Forecast next 30 days
    future_forecast = final_fit.forecast(all_data.values[-optimal_lag:], steps=30)
    
else:  # VECM
    # Refit VECM
    final_model = VECM(all_data, k_ar_diff=optimal_lag-1, coint_rank=coint_rank, deterministic='ci')
    final_fit = final_model.fit()
    
    # Forecast next 30 days
    future_forecast = final_fit.predict(steps=30)

# Create future dates
last_date = all_data.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30)
future_df = pd.DataFrame(future_forecast, index=future_dates, columns=all_data.columns)

# Summary statistics
print("\n📊 Future Forecast Summary (30 days):")
print("-"*60)

total_revenue = future_df.sum().sum()
daily_avg = future_df.sum(axis=1).mean()
top_product = future_df.sum().idxmax()
top_revenue = future_df.sum().max()

print(f"Total revenue (all products): ${total_revenue:,.0f}")
print(f"Average daily revenue: ${daily_avg:,.0f}")
print(f"Top product: {top_product} (${top_revenue:,.0f})")

# Growth rates
last_30_days = all_data.iloc[-30:].sum()
growth_rates = ((future_df.sum() - last_30_days) / last_30_days * 100).sort_values(ascending=False)

print("\n📈 Expected Growth Rates vs Last 30 Days:")
print("Top 5 Growing Products:")
for product, growth in growth_rates.head(5).items():
    print(f"  {product}: {growth:+.1f}%")

print("\nBottom 5 Products:")
for product, growth in growth_rates.tail(5).items():
    print(f"  {product}: {growth:+.1f}%")

# Visualize forecast
fig = go.Figure()

# Historical data (last 60 days)
historical = all_data.iloc[-60:]

# Plot top 5 products
top_5_products = future_df.sum().nlargest(5).index

for product in top_5_products:
    # Historical
    fig.add_trace(go.Scatter(
        x=historical.index,
        y=historical[product],
        mode='lines',
        name=f'{product} (Historical)',
        line=dict(width=2)
    ))
    
    # Forecast
    fig.add_trace(go.Scatter(
        x=future_df.index,
        y=future_df[product],
        mode='lines',
        name=f'{product} (Forecast)',
        line=dict(width=2, dash='dash')
    ))

fig.update_layout(
    title='Revenue Forecast - Top 5 Products (Next 30 Days)',
    xaxis_title='Date',
    yaxis_title='Revenue ($)',
    height=600,
    hovermode='x unified'
)

fig.show()

## 14. Final Summary and Recommendations

In [ ]:
print("="*80)
print("FINAL SUMMARY - RQ2 FORECASTING ANALYSIS")
print("="*80)

print(f"\n📊 MODEL SUMMARY:")
print(f"  Model Type: {model_type}")
print(f"  Optimal Lag: {optimal_lag}")
if model_type == 'VECM':
    print(f"  Cointegration Rank: {coint_rank}")
print(f"  Model Stability: {'✅ Stable' if is_stable else '❌ Unstable'}")
print(f"  Test MAPE: {mape:.2%}")
print(f"  Test RMSE: ${rmse:,.2f}")

print(f"\n📈 KEY INSIGHTS:")

# 1. Top performers
top_3 = future_df.sum().nlargest(3)
print(f"\n1. Top Revenue Generators (Next 30 days):")
for i, (product, revenue) in enumerate(top_3.items(), 1):
    print(f"   {i}. {product}: ${revenue:,.0f}")

# 2. Growth opportunities
high_growth = growth_rates[growth_rates > 10]
if len(high_growth) > 0:
    print(f"\n2. High Growth Products (>10% growth):")
    for product, growth in high_growth.head(3).items():
        print(f"   • {product}: +{growth:.1f}%")

# 3. Cannibalization findings
if n_significant > 0:
    print(f"\n3. Cannibalization Alert:")
    print(f"   ⚠️ {n_significant} significant causal relationships detected")
    print(f"   New products may be cannibalizing old product sales")
else:
    print(f"\n3. Cannibalization:")
    print(f"   ✅ No strong evidence of cannibalization")

# 4. Model diagnostics
print(f"\n4. Model Diagnostics:")
if model_type == 'VAR' and hasattr(model, 'test_normality'):
    if model.test_normality().pvalue > 0.05:
        print(f"   ✅ Residuals are normally distributed")
    else:
        print(f"   ⚠️ Residuals show non-normality")

print(f"\n💡 RECOMMENDATIONS:")
print(f"\n1. INVENTORY MANAGEMENT:")
print(f"   • Increase stock for: {', '.join(top_3.index[:3])}")
print(f"   • Monitor closely: Products with high growth potential")

print(f"\n2. MARKETING STRATEGY:")
if len(high_growth) > 0:
    print(f"   • Focus promotion on high-growth products")
    print(f"   • Leverage momentum of {high_growth.index[0]}")

declining = growth_rates[growth_rates < -5]
if len(declining) > 0:
    print(f"   • Review strategy for declining products: {', '.join(declining.index[:3])}")

print(f"\n3. PORTFOLIO OPTIMIZATION:")
if n_significant > 0:
    print(f"   • Address cannibalization between new and old products")
    print(f"   • Consider product differentiation strategies")
else:
    print(f"   • Product portfolio shows healthy independence")
    print(f"   • Continue with current product mix")

print(f"\n4. FORECAST UPDATES:")
print(f"   • Refit model weekly with new data")
print(f"   • Monitor forecast accuracy (current MAPE: {mape:.2%})")
print(f"   • Consider ensemble methods if MAPE > 15%")

print("\n" + "="*80)
print("✅ RQ2 ANALYSIS COMPLETE")
print("="*80)